<a href="https://colab.research.google.com/github/marielahdez/proyecto-4-ia/blob/main/proyecto_4_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar dependencias:

In [ ]:
!pip install pydub

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Conversión de archivos .mp4 a .wav y ajusta la frecuencia de muestreo a 16 kHz:

In [2]:
from pydub import AudioSegment
import librosa
import soundfile as sf
import os

# Directorios de entrada y salida
input_dir = "audios/"  # Cambia esta ruta a donde tienes los .mp4
output_dir = "processed_audios/"  # Carpeta para los audios .wav a 16 kHz
os.makedirs(output_dir, exist_ok=True)

# Procesar los archivos en el directorio
for filename in os.listdir(input_dir):
    if filename.endswith(".mp4"):  # Verifica que sea un archivo .mp4
        input_path = os.path.join(input_dir, filename)
        temp_wav_path = os.path.join(output_dir, filename.replace(".mp4", "_temp.wav"))
        final_wav_path = os.path.join(output_dir, filename.replace(".mp4", ".wav"))

        # Convertir de .mp4 a .wav usando pydub
        audio = AudioSegment.from_file(input_path, format="mp4")
        audio.export(temp_wav_path, format="wav")
        print(f"Convertido de mp4 a wav: {filename} -> {temp_wav_path}")

        # Convertir el .wav a 16 kHz usando librosa
        audio_data, original_sr = librosa.load(temp_wav_path, sr=None)  # Mantén la frecuencia original
        audio_16k = librosa.resample(audio_data, orig_sr=original_sr, target_sr=16000)  # Convertir a 16 kHz
        sf.write(final_wav_path, audio_16k, 16000)  # Guardar el archivo final
        print(f"Convertido a 16 kHz: {filename} -> {final_wav_path}")

        # Eliminar el archivo temporal
        os.remove(temp_wav_path)

print("Conversión completa.")

Convertido de mp4 a wav: audio17.mp4 -> processed_audios/audio17_temp.wav
Convertido a 16 kHz: audio17.mp4 -> processed_audios/audio17.wav
Convertido de mp4 a wav: audio13.mp4 -> processed_audios/audio13_temp.wav
Convertido a 16 kHz: audio13.mp4 -> processed_audios/audio13.wav
Convertido de mp4 a wav: audio15.mp4 -> processed_audios/audio15_temp.wav
Convertido a 16 kHz: audio15.mp4 -> processed_audios/audio15.wav
Convertido de mp4 a wav: audio14.mp4 -> processed_audios/audio14_temp.wav
Convertido a 16 kHz: audio14.mp4 -> processed_audios/audio14.wav
Convertido de mp4 a wav: audio7.mp4 -> processed_audios/audio7_temp.wav
Convertido a 16 kHz: audio7.mp4 -> processed_audios/audio7.wav
Convertido de mp4 a wav: audio3.mp4 -> processed_audios/audio3_temp.wav
Convertido a 16 kHz: audio3.mp4 -> processed_audios/audio3.wav
Convertido de mp4 a wav: audio1.mp4 -> processed_audios/audio1_temp.wav
Convertido a 16 kHz: audio1.mp4 -> processed_audios/audio1.wav
Convertido de mp4 a wav: audio16.mp4 ->

Transcribir los audios con Wav2Vec 2.0:

In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import soundfile as sf
import os

# Inicializar el modelo y el procesador de Wav2Vec 2.0
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53-spanish")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53-spanish")

# Carpeta con los audios
audio_dir = "processed_audios/"
output = []

# Iterar sobre los archivos de audio
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):  # Asegúrate de que los audios sean .wav
        audio_path = os.path.join(audio_dir, filename)

        # Cargar el audio
        audio_input, sr = sf.read(audio_path)

        # Procesar el audio para el modelo
        input_values = processor(audio_input, return_tensors="pt", sampling_rate=sr).input_values

        # Obtener la transcripción
        with torch.no_grad():
            logits = model(input_values).logits

        # Decodificar la salida a texto
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]

        # Guardar resultados
        output.append({"audio_filename": filename, "transcription": transcription})
        print(f"Transcripción de {filename}: {transcription}")

# Crear un DataFrame con las transcripciones
import pandas as pd
transcriptions = pd.DataFrame(output)
transcriptions.to_csv("transcriptions.csv", index=False)

print("Transcripción completa.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Transcripción de audio5.wav: se da entre los equipos que ganaron en la tapa de semifinales
Transcripción de audio14.wav: el este les dice pues como e david en espíritu le llama señor diciendo
Transcripción de audio13.wav: destaca por tener e algunas de las mejores vistas del desfiladero de la irmida osea es impresionante
Transcripción de audio4.wav: él les dice pues como david en espíritu le llama señor diciendo
Transcripción de audio15.wav: se da entre los equipos que e ganaron am en la tapa esemifinales
Transcripción de audio11.wav: depende  del este ministerio de educación cultura y deporte
Transcripción de audio16.wav: el examen estaba dividido en osea cuatro partes vocabulario y gramática e escritura audición y lectura
Transcripción de audio19.wav: su aspecto es el de un e nole oriental am como algo muy exótico
Transcripción de audio1.wav: depende del ministerio de educación cultura y deporte
Transcripción de audio18.wav: dehecho am ahora el nombre original o sea alude a esta vers

Revisar y etiquetar las transcripciones:

In [6]:
import pandas as pd
import re

# Cargar las transcripciones generadas (asegúrate de que el archivo tenga 'transcription' como columna)
transcriptions = pd.read_csv("transcriptions.csv")  # Cambia esta ruta a la ubicación de tu archivo

# Ver las primeras filas para inspeccionar
print(transcriptions.head())

# Lista de muletillas (fillers) que deseas detectar
fillers = ["eh", "e", "uh", "am", "o sea", "osea", "este", "bueno", "digo"]

# Función para etiquetar las transcripciones y mostrar el filler encontrado
def label_fillers(transcription):
    # Buscar cada filler usando expresiones regulares para asegurarnos de que sea una palabra completa
    for filler in fillers:
        # Usamos \b para asegurarnos de que sea una palabra completa (delimitada por espacios o puntuación)
        if re.search(r'\b' + re.escape(filler) + r'\b', transcription.lower()):
            return 1, filler  # Si encuentra, devuelve "1" y el filler detectado
    return 0, None  # Si no encuentra, devuelve "0" y sin filler

# Aplicar la función a cada transcripción para crear la columna 'has_fillers' y 'filler_found'
transcriptions['has_fillers'], transcriptions['filler_found'] = zip(*transcriptions['transcription'].apply(label_fillers))

# Verificar las etiquetas generadas y los fillers encontrados
print(transcriptions[['audio_filename', 'transcription', 'has_fillers', 'filler_found']].head())

# Guardar el archivo con las etiquetas
transcriptions.to_csv("metadata_with_labels.csv", index=False)
print("Archivo con etiquetas guardado como 'metadata_with_labels.csv'.")

  audio_filename                                      transcription
0     audio5.wav  se da entre los equipos que ganaron en la tapa...
1    audio14.wav  el este les dice pues como e david en espíritu...
2    audio13.wav  destaca por tener e algunas de las mejores vis...
3     audio4.wav  él les dice pues como david en espíritu le lla...
4    audio15.wav  se da entre los equipos que e ganaron am en la...
  audio_filename                                      transcription  \
0     audio5.wav  se da entre los equipos que ganaron en la tapa...   
1    audio14.wav  el este les dice pues como e david en espíritu...   
2    audio13.wav  destaca por tener e algunas de las mejores vis...   
3     audio4.wav  él les dice pues como david en espíritu le lla...   
4    audio15.wav  se da entre los equipos que e ganaron am en la...   

   has_fillers filler_found  
0            0         None  
1            1            e  
2            1            e  
3            0         None  
4            1 

Cargar y preprocesar los datos antes de entrenar el modelo:

In [12]:
from transformers import DistilBertTokenizer
from datasets import Dataset
import pandas as pd

# Cargar el archivo de etiquetas
metadata = pd.read_csv("metadata_with_labels.csv")  # Asegúrate de que el archivo esté en la ruta correcta

# Visualizar las primeras filas
print(metadata.head())

# Tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Función para tokenizar las transcripciones e incluir las etiquetas
def tokenize_function(examples):
    # Tokenizar las transcripciones
    tokenized = tokenizer(examples['transcription'], padding="max_length", truncation=True)
    # Añadir las etiquetas (has_fillers) como 'labels'
    tokenized['labels'] = examples['has_fillers']  # Aquí añadimos las etiquetas
    return tokenized

# Convertir el DataFrame a un Dataset compatible con HuggingFace
dataset = Dataset.from_pandas(metadata)

# Aplicar tokenización
dataset = dataset.map(tokenize_function, batched=True)

# Verifica que la tokenización haya funcionado
print(dataset[0])  # Imprimir el primer ejemplo para revisar la tokenización


  audio_filename                                      transcription  \
0     audio5.wav  se da entre los equipos que ganaron en la tapa...   
1    audio14.wav  el este les dice pues como e david en espíritu...   
2    audio13.wav  destaca por tener e algunas de las mejores vis...   
3     audio4.wav  él les dice pues como david en espíritu le lla...   
4    audio15.wav  se da entre los equipos que e ganaron am en la...   

   has_fillers filler_found  
0            0          NaN  
1            1            e  
2            1            e  
3            0          NaN  
4            1            e  


Map:   0%|          | 0/17 [00:00<?, ? examples/s]

{'audio_filename': 'audio5.wav', 'transcription': 'se da entre los equipos que ganaron en la tapa de semifinales', 'has_fillers': 0, 'filler_found': None, 'input_ids': [101, 7367, 4830, 4372, 7913, 3050, 1041, 15549, 6873, 2015, 10861, 25957, 10464, 2078, 4372, 2474, 11112, 2050, 2139, 16797, 2229, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Preparar los datos de entrenamiento y prueba:

In [13]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Dividir los datos en entrenamiento y prueba (80%/20%)
train_data, test_data = train_test_split(metadata, test_size=0.2, random_state=42)

# Convertir los datos en formato Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Aplicar tokenización a los datasets de entrenamiento y prueba (usando la misma función de tokenización)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Verifica los datos procesados
print(train_dataset[0])  # Mostrar un ejemplo procesado del conjunto de entrenamiento


Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

{'audio_filename': 'audio7.wav', 'transcription': 'su hermano menor fué fernando p de vulgaria', 'has_fillers': 0, 'filler_found': None, '__index_level_0__': 11, 'input_ids': [101, 10514, 11458, 2080, 2273, 2953, 11865, 2063, 9158, 1052, 2139, 29364, 2401, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Entrenar el modelo:

In [26]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch

# Cargar el modelo preentrenado de DistilBERT para clasificación
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # directorio para almacenar los resultados
    num_train_epochs=5,              # número de épocas
    per_device_train_batch_size=8,   # tamaño de batch de entrenamiento
    per_device_eval_batch_size=8,    # tamaño de batch de evaluación
    evaluation_strategy="epoch",     # evaluar al final de cada época
    save_strategy="epoch",           # Guardar al final de cada época
    logging_dir='./logs',            # directorio para logs
    learning_rate=1e-5,              # Ajustamos la tasa de aprendizaje
    warmup_steps=300,                # Ajustamos el número de warmup_steps
    load_best_model_at_end=True,     # Cargar el mejor modelo al final
    metric_for_best_model="eval_loss" # Métrica para determinar el mejor modelo
)

# Crear el Trainer
trainer = Trainer(
    model=model,                         # El modelo
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_dataset,         # Datos de entrenamiento
    eval_dataset=test_dataset,           # Datos de validación
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Se detendrá después de 2 épocas sin mejora
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
results = trainer.evaluate()
print(f"Results: {results}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.686043
2,No log,0.686045
3,No log,0.686039
4,No log,0.686091
5,No log,0.686065


Results: {'eval_loss': 0.6860387921333313, 'eval_runtime': 3.0937, 'eval_samples_per_second': 1.293, 'eval_steps_per_second': 0.323, 'epoch': 5.0}


Usar el modelo para predicciones:

In [21]:
# Función para predecir si un texto contiene muletillas
def predict_filler(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1)
    return "Tiene muletillas" if prediction == 1 else "No tiene muletillas"

# Ejemplo de predicción
new_transcription = "am, depende del ministerio de educación..."
result = predict_filler(new_transcription)
print(result)

No tiene muletillas
